In [ ]:
#load the files into your google drive and find them with this
#use !ls to see if if its there. if you dont know what that is go on google
import os
os.chdir('drive')
os.chdir('MyDrive')

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
#run this cell to load practice and test data
practiceData = pd.read_csv("201213-201718.csv") 
testData = pd.read_csv("201819.csv") 

In [ ]:
def DataStructures():
  #this dictionary is meant to be used to access awayTeamLastTen
  #some teams have repeated entries because of relocations/rebranding
  #New Jersey -> Brooklyn Nets and New Orleans Hornets - > Pelicans. Charlotte changed from Bobcats to hornets but doesnt change their letters
  teamDic = {"ATL":0,"BKN":1,"NJN":1,"BOS":2,"CHA":3,"CHI":4,"CLE":5,"DAL":6,"DEN":7,"DET":8,"GSW":9,"HOU":10,"IND":11,"LAC":12,"LAL":13,"MEM":14,"MIA":15,"MIL":16,"MIN":17,"NOP":18,"NOH":18,"NYK":19,"OKC":20,"ORL":21,"PHI":22,"PHX":23,"POR":24,"SAC":25,"SAS":26,"TOR":27,"UTA":28,"WAS":29}

  #(total home wins/ total homes games played)
  homeTeamWLAtHome = dict.fromkeys(teamDic,"0/0")

  #list of queues that holds the last 10 away games for the away team in terms of W:1, L:0
  #point of this is to use float(sum()/len()) to find win rate of last 10 away games for the away team
  #[1,1,1,0,0,0,0,0,0,0] = 0.3
  awayTeamLastTen = [[] for i in range(30)]

  #this is to calculate average point differential for both team (total/numberofgamesteamhasplayed)
  numberOfGamesTeamHasPlayed = dict.fromkeys(teamDic, 0)
  totalPD = dict.fromkeys(teamDic,0)

  return homeTeamWLAtHome,awayTeamLastTen,numberOfGamesTeamHasPlayed,totalPD

In [ ]:
def ExtractData(input):
  data = input

  tempTensor = []
  tempLabels = []

  homeTeamWLAtHome, awayTeamLastTen, numberOfGamesTeamHasPlayed, totalPD = DataStructures()

  for index, row in data.iterrows():
    
    #checks for if its a new season
    if (data.iloc[index,0] != data.iloc[index-1,0]) and index !=0 :
      homeTeamWLAtHome, awayTeamLastTen, numberOfGamesTeamHasPlayed, totalPD = DataStructures()

    matchup = row['MATCHUP'].split(" ")
    if matchup[1] == 'vs.':

      numberOfGamesTeamHasPlayed[matchup[0]] += 1
      numberOfGamesTeamHasPlayed[matchup[2]] += 1

      totalPD[matchup[0]] += row['PLUS_MINUS']
      totalPD[matchup[2]] += (row['PLUS_MINUS']*-1)

      didHomeTeamWin = -1

      #HOME TEAM WINS / AWAY TEAM LOSES
      if row['PLUS_MINUS'] > 0:
        #UPDATES home win/loss at home
        temp = homeTeamWLAtHome[matchup[0]]
        temp = temp.split("/")
        temp = str((int(temp[0])+1)) + "/" + str((int(temp[1])+1))         
        homeTeamWLAtHome[matchup[0]] = temp

        didHomeTeamWin = 1

        #LAST TEN AWAY GAMES FOR THE AWAY TEAM
        #APPEND 0 cause they lose
        if len(awayTeamLastTen[teamDic[matchup[2]]]) > 9:
          awayTeamLastTen[teamDic[matchup[2]]].pop(0)
          awayTeamLastTen[teamDic[matchup[2]]].append(0)
          
        else:
          awayTeamLastTen[teamDic[matchup[2]]].append(0)
          
      
      #HOME TEAM LOSES / AWAY TEAM WINS
      else:
        temp = homeTeamWLAtHome[matchup[0]]
        temp = temp.split("/")          
        temp = str((int(temp[0]))) + "/" + str((int(temp[1])+1))        
        homeTeamWLAtHome[matchup[0]] = temp

        didHomeTeamWin = 0

        #LAST TEN AWAY GAMES FOR THE AWAY TEAM
        #APPEND 1 cause they win
        if len(awayTeamLastTen[teamDic[matchup[2]]]) > 9:
          awayTeamLastTen[teamDic[matchup[2]]].pop(0)
          awayTeamLastTen[teamDic[matchup[2]]].append(1)
            
        else:
          awayTeamLastTen[teamDic[matchup[2]]].append(1)

      #prepping the tensor and labels

      homeTeam = matchup[0]
      awayTeam = matchup[2]

      homeTeamWinPercentAtHome = homeTeamWLAtHome[homeTeam].split("/")
      homeTeamWinPercentAtHome = float( int(homeTeamWinPercentAtHome[0])/int(homeTeamWinPercentAtHome[1]))

      awayTeamWinPercentForLastTen = float ( sum(awayTeamLastTen[teamDic[awayTeam]])/ len(awayTeamLastTen[teamDic[awayTeam]]) )

      homePD = float(totalPD[homeTeam]/numberOfGamesTeamHasPlayed[homeTeam])
      awayPD = float(totalPD[awayTeam]/numberOfGamesTeamHasPlayed[awayTeam]) 

      tempTensor.append( tf.constant( [teamDic[homeTeam],teamDic[awayTeam],homeTeamWinPercentAtHome,awayTeamWinPercentForLastTen,homePD,awayPD] ) )
      tempLabels.append(tf.constant([didHomeTeamWin]))

  tempTensor = tf.stack(tempTensor)
  tempLabels = tf.stack(tempLabels)

  return tempTensor,tempLabels  
  

In [ ]:
features, labels = ExtractData(practiceData)

In [ ]:
from tensorflow.keras import layers
model = tf.keras.Sequential([
  layers.Dense(6,input_dim=6,activation="relu"),
  layers.Dense(4,activation="relu"),
  layers.Dense(2,activation="relu"),
  #add more neurons as well see what happens

  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['binary_accuracy'])

model.fit(features, labels, epochs=300)
print("Done")